Added State method for parametric uncertainty (Inverted Pendulum Example)

Objectives:
Compare zero lvl set evolution and 3D value function for max(v_l, v_u) with max(v_l, v_1, v_2, ..., v_max), as well as disturbance method



In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 100

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
from matplotlib import MatplotlibDeprecationWarning
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
import matplotlib.pyplot as plt
import jax.numpy as jnp
import jax
import numpy as np

import matplotlib
import pickle as pkl

from scipy.interpolate import interp1d
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

import hj_reachability as hj
import utils

Setup Problem (dynamics, environment)

In [ ]:
class inv_pend(hj.ControlAndDisturbanceAffineDynamics):
    STATES = ["\theta", "\dot \theta"]
    CONTROLS = ["\tau"]
    DISTURBANCES = ["\nu"]
    def __init__(self, uMax=10, dMin=1, dMax=2, params=None,
                 control_mode="min", disturbance_mode="max",
                 control_space=None, disturbance_space=None
                 ):
        self.uMax = jnp.array(uMax)
        self.dMin = jnp.array(dMin)
        self.dMax = jnp.array(dMax)
        if control_space is None:
            control_space = hj.sets.Box(lo=jnp.array([-self.uMax]), hi=jnp.array([self.uMax]))
        if disturbance_space is None:
            disturbance_space = hj.sets.Box(lo=jnp.array([self.dMin]), hi=jnp.array([self.dMax]))
        super().__init__(control_mode, disturbance_mode, control_space, disturbance_space)

    #Don't need to define control jacobian as there's only 1 control input
    def control_jacobian(self, state, time):
        return jnp.array([
            [0],
            [1],
            [0],
        ])
    
    def disturbance_jacobian(self, state, time):
        return jnp.array([
            [0],
            [0],
            [0],
        ])
    
    def open_loop_dynamics(self, state, time):
        return jnp.array([state[1], -9.81*state[2]*jnp.sin(state[0]+jnp.pi), 0])

In [ ]:
class inv_pend_dist(hj.ControlAndDisturbanceAffineDynamics):
    STATES = ["\theta", "\dot \theta"]
    CONTROLS = ["\tau"]
    DISTURBANCES = ["\nu"]
    def __init__(self, uMax=10, dMin=1, dMax=2, params=None,
                 control_mode="min", disturbance_mode="max",
                 control_space=None, disturbance_space=None
                 ):
        self.uMax = jnp.array(uMax)
        self.dMin = jnp.array(dMin)
        self.dMax = jnp.array(dMax)
        if control_space is None:
            control_space = hj.sets.Box(lo=jnp.array([-self.uMax]), hi=jnp.array([self.uMax]))
        if disturbance_space is None:
            disturbance_space = hj.sets.Box(lo=jnp.array([self.dMin]), hi=jnp.array([self.dMax]))
        super().__init__(control_mode, disturbance_mode, control_space, disturbance_space)

    #Don't need to define control jacobian as there's only 1 control input
    def control_jacobian(self, state, time):
        return jnp.array([
            [0],
            [1],
        ])
    
    def disturbance_jacobian(self, state, time):
        return jnp.array([
            [0],
            [-9.81*jnp.sin(state[0]+jnp.pi)],
        ])
    
    def open_loop_dynamics(self, state, time):
        return jnp.array([state[1], 0])

In [ ]:
dyn = inv_pend(uMax=3.0, dMin=1.0, dMax=1.0)
dyn_dist = inv_pend_dist(uMax=3.0, dMin=1.0, dMax=2.0)

In [ ]:
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(
    domain = hj.sets.Box(lo=np.array([-jnp.pi, -20, 1]), hi=np.array([jnp.pi, 20, 2])),
    shape = (111, 111, 5),
    # Boundary conditions matter for accuracy on the edge of the domain grid.
    # E.g. if we have an angle theta \in [0, 2pi] then we want a periodic boundary in that dim.
    periodic_dims=0,
    #boundary_conditions=(hj.boundary_conditions.extrapolate_away_from_zero,
    #                    hj.boundary_conditions.extrapolate_away_from_zero)
)
grid_dist = hj.Grid.from_lattice_parameters_and_boundary_conditions(
    domain = hj.sets.Box(lo=np.array([-jnp.pi, -20]), hi=np.array([jnp.pi, 20])),
    shape = (111, 111),
    # Boundary conditions matter for accuracy on the edge of the domain grid.
    # E.g. if we have an angle theta \in [0, 2pi] then we want a periodic boundary in that dim.
    periodic_dims=0,
    #boundary_conditions=(hj.boundary_conditions.extrapolate_away_from_zero,
    #                    hj.boundary_conditions.extrapolate_away_from_zero)
)
x_init = [-jnp.pi,0]
direction = 'backward'

In [ ]:
target = jnp.array([[-0.1, 0.1], [-1, 1], [1,2.5]])
target_dist = jnp.array([[-0.1, 0.1], [-1, 1]])
sdf = utils.build_target_sdf(target, 0.1)
sdf_dist = utils.build_target_sdf(target_dist, 0.1)

sdf_values = hj.utils.multivmap(sdf, jnp.arange(grid.ndim))(grid.states)
sdf_values_dist = hj.utils.multivmap(sdf_dist, jnp.arange(grid_dist.ndim))(grid_dist.states)

sdf_values = hj.shapes.shape_ellipse(grid, [0,0,0], [0.1, 0.05, None])
sdf_values_dist = hj.shapes.shape_ellipse(grid_dist, [0,0], [0.1, 0.05])

In [ ]:
cf = plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], sdf_values[:,:,0].T, levels=100)
plt.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1], sdf_values[:,:,0].T, levels=[0], colors='k', linewidths=3.0)
# add colorbar
plt.colorbar(cf)
#hj.viz.visFunc(grid_dist, sdf_values[:,:,0])
#hj.viz.visFunc(grid_dist, sdf_values_dist)

In [ ]:
hamiltonian_postprocessor = lambda x : jnp.minimum(x,0) #BRT
obstacles = None

accuracy = "very_high"

artificial_dissipation_scheme = hj.artificial_dissipation.global_lax_friedrichs

x_init_for_solving = None

identity = lambda *x: x[-1]
if 'hamiltonian_postprocessor' not in locals():
    hamiltonian_postprocessor=identity

if 'value_postprocessor' not in locals():
    value_postprocessor=identity

solver_settings = hj.SolverSettings.with_accuracy(
    accuracy=accuracy,
    x_init=x_init_for_solving,
    artificial_dissipation_scheme=artificial_dissipation_scheme,
    hamiltonian_postprocessor=hamiltonian_postprocessor,
    value_postprocessor=value_postprocessor,
)
times = np.linspace(0, -6, 600)

In [ ]:
times, target_values = hj.solve(
   solver_settings, dyn, grid, times, initial_values=sdf_values)

In [ ]:
times, target_values_dist = hj.solve(
   solver_settings, dyn_dist, grid_dist, times, initial_values=sdf_values_dist)

In [ ]:
(target_values[-1] >= 0).sum() / target_values[-1].size

In [ ]:
v_l = target_values[:, :, :, 0]
v_u = target_values[:, :, :, -1]

v_int_ext = jnp.maximum(v_l,v_u)
v_int_all = jnp.max(target_values, axis=3)

In [ ]:
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

vmax = np.abs(target_values[0]).max()
cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
                 v_int_ext[0].T)
#Extrema
cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           v_int_ext[0,:,:].T, levels=[0], colors='green', linewidths=2)
#All
cont2 = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           v_int_all[0,:,:].T, levels=[0], colors='black', linewidths=2)
#Disturbance
cont3 = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           target_values_dist[0,:,:].T, levels=[0], colors='red', linewidths=2)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title(f'HJR time $t=0$')
ax.set_xlabel('$y$ (Horizontal)')
ax.set_ylabel('$z$ (Vertical)')
tx = ax.set_title(f'$v_y=0, v_z=0$, HJR time $t=0$')

# Update function to draw contours for a given idi value
def update(idi):
    global cont, cont2, cont3
    arr = v_int_ext[idi, :, :].T
    arr2 = v_int_all[idi, :, :].T
    arr3 = target_values_dist[idi, :, :].T
    vmax = np.abs(arr).max()
    # ax.clear()
    cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], arr) #, vmax=vmax, vmin=-vmax)
    cont.collections[0].remove()
    cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr, levels=[0], colors='green')
    cont2.collections[0].remove()
    cont2 = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr2, levels=[0], colors='black')
    cont3.collections[0].remove()
    cont3 = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr3, levels=[0], colors='red')
    # cax.cla()
    tx.set_text('HJR time t={:.2f}'.format(np.abs(times[idi].item())))

# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times))[::5])
plt.close()
HTML(ani.to_jshtml())
# ani.save('addstate_dist.mp4')

In [ ]:
class inv_pend_test(hj.ControlAndDisturbanceAffineDynamics):
    def __init__(self, uMax=10, dMin=1, dMax=2, params=None,
                 control_mode="min", disturbance_mode="max",
                 control_space=None, disturbance_space=None
                 ):
        self.uMax = jnp.array(uMax)
        self.dMin = jnp.array(dMin)
        self.dMax = jnp.array(dMax)
        if control_space is None:
            control_space = hj.sets.Box(lo=jnp.array([-self.uMax]), hi=jnp.array([self.uMax]))
        if disturbance_space is None:
            disturbance_space = hj.sets.Box(lo=jnp.array([
                self.dMin]), hi=jnp.array([self.dMax]))
        super().__init__(control_mode, disturbance_mode, control_space, disturbance_space)

    #Don't need to define control jacobian as there's only 1 control input
    def control_jacobian(self, state, time):
        return jnp.array([
            [0],
            [1],
        ])
    
    def disturbance_jacobian(self, state, time):
        return jnp.array([
            [0],
            [-9.81*jnp.sin(state[0]+jnp.pi)],
        ])
    
    def open_loop_dynamics(self, state, time):
        return jnp.array([state[1], 0.0])

In [ ]:
x_init = jnp.array([-jnp.pi, 0.0])
dyn_orig = inv_pend_test(uMax=3.0, dMin=2.0, dMax=2.0) #Lets assume l = 1
dyn_alt = inv_pend_test(uMax=3.0, dMin=1.0, dMax=1.0)
t_traj, x_traj, contr_seq, distr_seq = dyn_orig.backtrack_trajectory(grid_dist, x_init, times, v_int_all)
t_traj2, x_traj2, contr_seq2, distr_seq2 = dyn_alt.backtrack_trajectory(grid_dist, x_init, times, v_int_all)
t_traj3, x_traj3, contr_seq3, distr_seq3 = dyn_orig.backtrack_trajectory(grid_dist, x_init, times, target_values[...,-1])
t_traj4, x_traj4, contr_seq4, distr_seq4 = dyn_alt.backtrack_trajectory(grid_dist, x_init, times, target_values[...,0])
t_trajd, x_trajd, contr_seqd, distr_seqd = dyn_dist.backtrack_trajectory(grid_dist, x_init, times, target_values_dist)

In [ ]:
# import mpatches
import matplotlib.patches as mpatches
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel(r'$\theta$', fontsize=12)  # theta for the x-axis
ax.set_ylabel(r'$\dot{\theta}$', fontsize=12)  # dot theta for the y-axis
plt.subplots_adjust(left=0.135, right=0.95, top=0.925, bottom=0.1)
def draw_rectangle(angle, color, height, label=None, alpha=1.0):
    rect = mpatches.Rectangle((.05, 0), 0.1, height, angle=angle * 180 / np.pi, color=color, label=label, alpha=alpha)
    return rect
# Update function to draw contours for a given idi value
def update(idi):
    # Remove all patches
    for patch in ax.patches:
        patch.remove()
    ax.add_patch(draw_rectangle(x_traj[0,idi], color='green', height=0.5, label= 'extrema'))
    ax.add_patch(draw_rectangle(x_traj2[0,idi], color='black', height=1.0, label= 'extrema (high)'))
    ax.add_patch(draw_rectangle(x_trajd[0,idi], color='red', height=1/distr_seqd[0][idi-1], label= 'dist.'))
    ax.add_patch(draw_rectangle(x_traj3[0,idi], color='green', height=0.5, label= 'extrema', alpha=0.5))
    ax.add_patch(draw_rectangle(x_traj4[0,idi], color='black', height=1.0, label= 'extrema (high)', alpha=0.5))    
    ax.add_patch(mpatches.Rectangle((.05, 0), 0.1, 1, linewidth=1, edgecolor='blue', linestyle=':', facecolor='none', label= 'target'))
    ax.set_title(f"Time elapsed: {t_traj[idi]:.2f} seconds")
    if idi == times[-1]:  # Assuming you want to add the legend once, do it on the first frame
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))  # Removing duplicates if any
        ax.legend(by_label.values(), by_label.keys())


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=reversed(range(len(times))))
plt.close()
HTML(ani.to_jshtml())
# ani.save('ip_online_param.mp4')

In [ ]:
x_init = jnp.array([-jnp.pi, 0.0])
dyn_orig = inv_pend_test(uMax=3.0, dMin=2.0, dMax=2.0) #Lets assume l = 1
dyn_alt = inv_pend_test(uMax=3.0, dMin=1.0, dMax=1.0)
t_traj, x_traj, contr_seq, distr_seq = dyn_orig.backtrack_trajectory(grid_dist, x_init, times, v_int_ext)
t_traj2, x_traj2, contr_seq2, distr_seq2 = dyn_alt.backtrack_trajectory(grid_dist, x_init, times, v_int_ext)
t_traj3, x_traj3, contr_seq3, distr_seq3 = dyn_orig.backtrack_trajectory(grid_dist, x_init, times, target_values[...,-1])
t_traj4, x_traj4, contr_seq4, distr_seq4 = dyn_alt.backtrack_trajectory(grid_dist, x_init, times, target_values[...,0])
t_trajd, x_trajd, contr_seqd, distr_seqd = dyn_dist.backtrack_trajectory(grid_dist, x_init, times, target_values_dist)

In [ ]:
# import mpatches
import matplotlib.patches as mpatches
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel(r'$\theta$', fontsize=12)  # theta for the x-axis
ax.set_ylabel(r'$\dot{\theta}$', fontsize=12)  # dot theta for the y-axis
plt.subplots_adjust(left=0.135, right=0.95, top=0.925, bottom=0.1)
def draw_rectangle(angle, color, height, label=None, alpha=1.0):
    rect = mpatches.Rectangle((.05, 0), 0.1, height, angle=angle * 180 / np.pi, color=color, label=label, alpha=alpha)
    return rect
# Update function to draw contours for a given idi value
def update(idi):
    # Remove all patches
    for patch in ax.patches:
        patch.remove()
    ax.add_patch(draw_rectangle(x_traj[0,idi], color='green', height=0.5, label= 'extrema'))
    ax.add_patch(draw_rectangle(x_traj2[0,idi], color='black', height=1.0, label= 'extrema (high)'))
    ax.add_patch(draw_rectangle(x_trajd[0,idi], color='red', height=1/distr_seqd[0][idi-1], label= 'dist.'))
    ax.add_patch(draw_rectangle(x_traj3[0,idi], color='green', height=0.5, label= 'extrema', alpha=0.5))
    ax.add_patch(draw_rectangle(x_traj4[0,idi], color='black', height=1.0, label= 'extrema (high)', alpha=0.5))    
    ax.add_patch(mpatches.Rectangle((.05, 0), 0.1, 1, linewidth=1, edgecolor='blue', linestyle=':', facecolor='none', label= 'target'))
    ax.set_title(f"Time elapsed: {t_traj[idi]:.2f} seconds")
    if idi == times[-1]:  # Assuming you want to add the legend once, do it on the first frame
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))  # Removing duplicates if any
        ax.legend(by_label.values(), by_label.keys())


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=reversed(range(len(times))))
plt.close()
HTML(ani.to_jshtml())
# ani.save('ip_online_param.mp4')

## Now using Experiment Wrapper

In [ ]:
from experiment_wrapper import RolloutTrajectory, StateSpaceExperiment, TimeSeriesExperiment
from cbf_opt import ControlAffineDynamics, BatchedDynamics

In [ ]:
class ReachController:
    def __init__(self, grid, tv_vf, dynamics):
        self.grid = grid
        self.tv_vf = tv_vf
        self.dynamics = dynamics

    def _line_search(self, x):
        upper = len(self.tv_vf)
        lower = 0

        while upper > lower:
            mid = (upper + lower) // 2
            val = self.grid.interpolate(self.tv_vf[mid], x[0])

            lower, upper = jnp.where(
                val < -1e-4, jnp.array([lower, mid]), jnp.array([mid + 1, upper])
            )
        return mid

    def get_nominal_control(self, x, t):
        idx = self._line_search(x)
        grad_vals = self.grid.grad_values(self.tv_vf[idx])
        grad_val = self.grid.interpolate(grad_vals, x[0])
        return self.dynamics.optimal_control_and_disturbance(x[0], 0.0, grad_val)[0]

In [ ]:
dyn_new = inv_pend_test(uMax=3.0, dMin=2.0, dMax=2.0) #Lets assume l = 0.5

In [ ]:
extrema_controller = ReachController(grid_dist, v_int_ext, dyn_new).get_nominal_control

In [ ]:
all_param_controller = ReachController(grid_dist, v_int_all, dyn_orig).get_nominal_control

In [ ]:
dist_controller = ReachController(grid_dist, target_values_dist, dyn_orig).get_nominal_control

In [ ]:
class InvertedPendulum(ControlAffineDynamics):
    """
    Simplified dynamics, and we need to convert controls from phi to tan(phi)"""
    STATES = ["\theta", "\dot \theta"]
    CONTROLS = ["\tau"]
    DISTURBANCES = ["\nu"]
    def __init__(self, params, test=True, **kwargs):
        self.g = params.get("g", 9.81)
        super().__init__(params, **kwargs)
    
    def open_loop_dynamics(self, state, time: float = 0.0):
        return jnp.array([state[1], 0.0])

    def control_matrix(self, state, time: float = 0.0):
        return jnp.array([[0.0], [1.0]])
    
    def state_jacobian(self, state, control, disturbance = None, time: float = 0.0):
        return jax.jacfwd(lambda x: self.__call__(x, control, disturbance, time))(state)
    
    def disturbance_matrix(self, state, time: float = 0):
        return jnp.array([[0.0], [-self.g * jnp.sin(state[0] + jnp.pi)]])
    

In [ ]:
x0 = jnp.array([jnp.pi, 0.0])
experiment = RolloutTrajectory('quad', start_x=x0, n_sims_per_start=1, t_sim=6.0)

In [ ]:
dyn = BatchedDynamics(InvertedPendulum(params={"g": 9.81}, dt=0.06))

import logging
logging.getLogger('cbf_opt').setLevel(level=logging.ERROR)
results_df = experiment.run(dyn, {"extrema": extrema_controller, "all_params": all_param_controller}, 
                            disturbances={"low": lambda x, t: jnp.array([1.0]), 
                                          "med": lambda x, t: jnp.array([1.5]), 
                                          "high": lambda x, t: jnp.array([2.0])})

In [ ]:
# import mpatches
import matplotlib.patches as mpatches

# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel(r'$\theta$', fontsize=12)  # theta for the x-axis
ax.set_ylabel(r'$\dot{\theta}$', fontsize=12)  # dot theta for the y-axis
plt.subplots_adjust(left=0.135, right=0.95, top=0.925, bottom=0.1)
xvals = []
for dist in results_df.disturbance.unique():
    vals = results_df[(results_df.controller == "extrema") & (results_df.disturbance == dist) & (results_df.measurement == "\theta")]
    xvals.append(vals.value.to_numpy())
xvals = np.array(xvals)
times_exp = vals.t.to_numpy()

alphas = [0.2, 0.5, 1.0]
heights = [1.0, 2/3, 0.5]
def draw_rectangle(angle, color, height, label=None, alpha=1.0):
    rect = mpatches.Rectangle((.05, 0), 0.1, height, angle=angle * 180 / np.pi, color=color, label=label, alpha=alpha)
    return rect
# Update function to draw contours for a given idi value
def update(idi):
    # Remove all patches
    for patch in ax.patches:
        patch.remove()
    for i, xval in enumerate(xvals):
        ax.add_patch(draw_rectangle(xval[idi], color='green', height=heights[i], label= 'extrema', alpha=alphas[i]))
    ax.add_patch(mpatches.Rectangle((.05, 0), 0.1, 1, linewidth=1, edgecolor='blue', linestyle=':', facecolor='none', label= 'target'))
    ax.set_title(f"Time elapsed: {times_exp[idi]:.2f} seconds")
    if idi == 0:  # Assuming you want to add the legend once, do it on the first frame
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))  # Removing duplicates if any
        ax.legend(by_label.values(), by_label.keys())


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times_exp)))
plt.close()
HTML(ani.to_jshtml())
# ani.save('ip_online_param.mp4')

In [ ]:
x0 = jnp.array([jnp.pi, 0.0])
experiment = RolloutTrajectory('quad', start_x=x0, n_sims_per_start=1, t_sim=10.0)

In [ ]:
dyn = BatchedDynamics(InvertedPendulum(params={"g": 9.81}, dt=0.01))

import logging
logging.getLogger('cbf_opt').setLevel(level=logging.ERROR)
results_df = experiment.run(dyn, {"extrema": extrema_controller}, 
                            disturbances={"low": lambda x, t: jnp.array([1.0]), 
                                          "med": lambda x, t: jnp.array([1.5]), 
                                          "high": lambda x, t: jnp.array([2.0])})

In [ ]:
# import mpatches
import matplotlib.patches as mpatches

# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel(r'$\theta$', fontsize=12)  # theta for the x-axis
ax.set_ylabel(r'$\dot{\theta}$', fontsize=12)  # dot theta for the y-axis
plt.subplots_adjust(left=0.135, right=0.95, top=0.925, bottom=0.1)
xvals = []
for dist in results_df.disturbance.unique():
    vals = results_df[(results_df.controller == "extrema") & (results_df.disturbance == dist) & (results_df.measurement == "\theta")]
    xvals.append(vals.value.to_numpy())
xvals = np.array(xvals)
times_exp = vals.t.to_numpy()

alphas = [0.2, 0.5, 1.0]
heights = [1.0, 2/3, 0.5]
def draw_rectangle(angle, color, height, label=None, alpha=1.0):
    rect = mpatches.Rectangle((.05, 0), 0.1, height, angle=angle * 180 / np.pi, color=color, label=label, alpha=alpha)
    return rect
# Update function to draw contours for a given idi value
def update(idi):
    # Remove all patches
    for patch in ax.patches:
        patch.remove()
    for i, xval in enumerate(xvals):
        ax.add_patch(draw_rectangle(xval[idi], color='green', height=heights[i], label= 'extrema', alpha=alphas[i]))
    ax.add_patch(mpatches.Rectangle((.05, 0), 0.1, 1, linewidth=1, edgecolor='blue', linestyle=':', facecolor='none', label= 'target'))
    ax.set_title(f"Time elapsed: {times_exp[idi]:.2f} seconds")
    if idi == 0:  # Assuming you want to add the legend once, do it on the first frame
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))  # Removing duplicates if any
        ax.legend(by_label.values(), by_label.keys())


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times_exp)))
plt.close()
# HTML(ani.to_jshtml())
# ani.save('ip_online_param.mp4')